In [1]:
# import libraries
import pandas as pd

In [2]:
# read in the data
data_df = pd.read_csv('../data/processed_csv/project3_raw_complete.csv')
data_df

,AGE2,IRMARIT,SERVICE,HEALTH,MOVSINPYR2,SEXATRACT,SEXIDENT,DIFFTHINK,DIFFWALK,DIFFDRESS,...,PNRANYLIF,TRQANYLIF,STMANYLIF,SEDANYLIF,DSTNRV30,DSTHOP30,DSTRST30,DSTCHR30,DSTEFF30,DSTNGD30
0,15,1,2.0,2.0,0.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,2.0,4.0
1,17,1,2.0,2.0,0.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0
2,14,4,2.0,2.0,0.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,4.0,5.0,5.0,5.0,5.0,5.0
3,8,4,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,...,2.0,1.0,1.0,2.0,4.0,5.0,5.0,5.0,5.0,5.0
4,14,1,2.0,2.0,0.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,2.0,2.0,4.0,5.0,5.0,4.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120538,15,1,2.0,2.0,0.0,1.0,1.0,2.0,2.0,2.0,...,1.0,2.0,1.0,2.0,4.0,5.0,5.0,5.0,5.0,5.0
120539,10,4,2.0,4.0,3.0,2.0,1.0,1.0,2.0,2.0,...,1.0,1.0,1.0,1.0,3.0,3.0,2.0,2.0,2.0,4.0
120540,16,4,2.0,2.0,0.0,5.0,2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,1.0,4.0,5.0,5.0,5.0,4.0,5.0
120541,12,4,2.0,3.0,1.0,1.0,1.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,5.0,4.0,5.0,5.0,5.0,5.0


In [3]:
# Assign X (data) and y (target)
X = data_df.drop(columns=['CG30EST'])
y = data_df['CG30EST'].values
print(X.shape, y.shape)

(120543, 63) (120543,)


In [4]:
from sklearn.preprocessing import LabelEncoder
# Step 1: Label-encode data set
encoded_y = LabelEncoder().fit(y).transform(y)

In [5]:
codes_df = pd.DataFrame({"Original class": y, "Encoded label": encoded_y})

In [6]:
# show original clas with encoded labels
codes_df.drop_duplicates()


,Original class,Encoded label
0,93.0,7
1,91.0,6
3543,6.0,5
4909,2.0,1
6987,4.0,3
12647,5.0,4
21571,1.0,0
37475,3.0,2


In [7]:
# Use sklearn's `train_test_split` to split the data into training and testing
from sklearn.model_selection import train_test_split

# split is random (shuffled) automatically
X_train_test, X_val, y_train_test, y_val = train_test_split(X, encoded_y, random_state=42, test_size=0.1)
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, random_state=42, test_size=0.2)

# Logistic Regression

In [8]:
# Create a Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=100000)

In [9]:
# Fit (train) or model using the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=100000)

In [10]:
# print the R2 score before adjusting parameters
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")
# print(f"Validation Data Score: {classifier.score(X_val, y_val)}")

Training Data Score: 0.9993547643737758
Testing Data Score: 0.9994008664393031


In [11]:
# make a prediction on the validation data before adjusting parameters
predictions = classifier.predict(X_val)
pd.DataFrame({"Prediction": predictions, "Actual": y_val}).head(15)

,Prediction,Actual
0,7,7
1,7,7
2,7,7
3,7,7
4,6,6
5,7,7
6,6,6
7,6,6
8,6,6
9,7,7


In [12]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'penalty': ['L1', 'L2', 'none']}
grid = GridSearchCV(classifier, param_grid, verbose=True, n_jobs=-2)

In [13]:
# Fit the model using the grid search estimator. 
grid.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-2)]: Done  60 out of  60 | elapsed: 59.2min finished


GridSearchCV(estimator=LogisticRegression(max_iter=100000), n_jobs=-2,
             param_grid={'C': [1, 5, 10, 50], 'penalty': ['L1', 'L2', 'none']},
             verbose=True)

In [14]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'penalty': 'none'}


In [15]:
# List the best score
print(grid.best_score_)

0.9990667127549256


In [16]:
# print the R2 score before adjusting parameters
print(f"Training Data Score: {grid.score(X_train, y_train)}")
print(f"Testing Data Score: {grid.score(X_test, y_test)}")
# print(f"Validation Data Score: {grid.score(X_val, y_val)}")

Training Data Score: 0.9994008526327918
Testing Data Score: 0.9993547792423265


In [17]:
# make a prediction on the validation data
predictions = grid.predict(X_val)
pd.DataFrame({"Prediction": predictions, "Actual": y_val}).head(15)

,Prediction,Actual
0,7,7
1,7,7
2,7,7
3,7,7
4,6,6
5,7,7
6,6,6
7,6,6
8,6,6
9,7,7


In [18]:
import joblib
# save/ export the model
joblib.dump(grid, '../Machine_learning_models/Smoking_LogRegr_fulldataset.smd')
print("Model is saved.")

Model is saved.


In [21]:
load_model = joblib.load('../Machine_learning_models/Smoking_LogRegr_fulldataset.smd')

In [22]:
load_model.predict(X_val)

array([7, 7, 7, ..., 7, 6, 6])

# Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200)

In [24]:
print(f"Training Data Score: {rf.score(X_train, y_train)}")
print(f"Testing Data Score: {rf.score(X_test, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9994008664393031


In [25]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_

# We can sort the features by their importance
sorted(zip(rf.feature_importances_, data_df.columns), reverse=True)

[(0.2674064738769352, 'CIGTRY'),
 (0.25300586965044, 'CIG100LF'),
 (0.20864180918835282, 'CIGEVER'),
 (0.18682143129227213, 'CIGDLYMO'),
 (0.024717671065791726, 'ALCUS30D'),
 (0.010625807344324658, 'ALCEVER'),
 (0.007217242518963189, 'MJEVER'),
 (0.006615774843930454, 'CG30EST'),
 (0.00449400592064161, 'ALCDAYS'),
 (0.004198445504117483, 'ALCYRTOT'),
 (0.004119139310831624, 'ALCREC'),
 (0.003969619849952927, 'MESC'),
 (0.0030963360068676418, 'AGE2'),
 (0.002759166032541311, 'HEREVER'),
 (0.002482517548388608, 'PSILCY'),
 (0.001572754461626421, 'METHAMEVR'),
 (0.0012456504619727778, 'IRMARIT'),
 (0.0011392819807958304, 'PNRANYLIF'),
 (0.0009804599538150846, 'LGAS'),
 (0.0008246748227645565, 'IRPINC3'),
 (0.0007494443173058704, 'TRQANYLIF'),
 (0.0007465964667163535, 'INHALOTH'),
 (0.00032658235685181336, 'WRKSTATWK2'),
 (0.00022367677618397532, 'IRWRKSTAT'),
 (0.0001870410364060273, 'STMANYLIF'),
 (0.0001673935331261748, 'IREDUHIGHST2'),
 (0.0001434160784214108, 'HEALTH'),
 (0.0001231352

In [26]:
# make a prediction on the validation data
rf_predictions = rf.predict(X_val)
pd.DataFrame({"Prediction": predictions, "Actual": y_val}).head(15)

,Prediction,Actual
0,7,7
1,7,7
2,7,7
3,7,7
4,6,6
5,7,7
6,6,6
7,6,6
8,6,6
9,7,7


In [27]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
rf = RandomForestClassifier()
rf_param_grid = {'n_estimators': [100, 200, 500],
                'max_features': [10, 20, 50]}
grid_rf = GridSearchCV(rf, rf_param_grid, verbose=True, n_jobs=-2)

In [28]:
# Fit the model using the grid search estimator. 
grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  4.5min finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-2,
             param_grid={'max_features': [10, 20, 50],
                         'n_estimators': [100, 200, 500]},
             verbose=True)

In [29]:
# List the best parameters for this dataset
print(grid_rf.best_params_)

{'max_features': 10, 'n_estimators': 100}


In [30]:
# List the best score
print(grid_rf.best_score_)

0.9993547643737758


In [31]:
# print the R2 score before adjusting parameters
print(f"Training Data Score: {grid_rf.score(X_train, y_train)}")
print(f"Testing Data Score: {grid_rf.score(X_test, y_test)}")
# print(f"Validation Data Score: {grid.score(X_val, y_val)}")

Training Data Score: 0.999988477935246
Testing Data Score: 0.9994008664393031


In [32]:
# make a prediction on the validation data
predictions_rf = grid_rf.predict(X_val)
pd.DataFrame({"Prediction": predictions_rf, "Actual": y_val}).head(15)

,Prediction,Actual
0,7,7
1,7,7
2,7,7
3,7,7
4,6,6
5,7,7
6,6,6
7,6,6
8,6,6
9,7,7


In [33]:
# save/ export the model
joblib.dump(grid_rf, '../Machine_learning_models/Smoking_RandomForest_fulldataset.smd')
print("Model is saved.")

Model is saved.


In [34]:
load_model_rf = joblib.load('../Machine_learning_models/Smoking_RandomForest_fulldataset.smd')

In [35]:
load_model_rf.predict(X_val)

array([7, 7, 7, ..., 7, 6, 6])